__Author: Rajendra Thapa__

This notebook creates a tracking system for the nbd of user-chosen atoms during a MD simulation. It was created to work with the dump file from (DL-POLY) but can be easily modified to work with any other software output files. 

Notes:
- This is a LiNbO$_3$ system

In [3]:
import pyprind 
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
def periodic_distance(atom, nbd, box):
    '''Inputs:
        atom = coordinate of a single reference atom
        nbd = array of coordinates of multiple atoms
        box = length of the box
       Outputs:
        dis = distance between atom and nbd
        len(dis) == len(nbd)
    '''
    delta = np.abs(atom-nbd)
    delta = np.where(delta> 0.5*box, delta-box, delta)
    dis = np.sqrt((delta**2).sum(axis=-1))
    return dis

In [1]:
file_paths = ['../A0_HISTORY', '../A1_HISTORY']  

# Create a list of index of the particles that you want to track the nbd for
# Take care of the spaces before and after the strings

Nb_index = [' 36746 ',' 36252 ' ]
# Create lists to store the coordinates of the Nb centers
Nb_I, Nb_II = [], []
# Open the file and search for the required strings
for file_path in file_paths:
    with open(file_path, 'r') as file:
        previous_line = None
        found = False
        for line in file:
            if "Nb5+" in line and Nb_index[0] in line:
                if previous_line is not None:
                    found = True
                else:
                    found = False
            elif found:
                Nb_I.append(line.split())  # Print the line following the match
                found = False
            previous_line = line

        if found:
            print("No line following the last match")
        
        
        
# Open the file and search for the required strings
for file_path in file_paths:
    with open(file_path, 'r') as file:
        previous_line = None
        found = False
        for line in file:
            if "Nb5+" in line and Nb_index[1] in line:
                if previous_line is not None:
                    found = True
                else:
                    found = False
            elif found:
                Nb_II.append(line.split())  # Print the line following the match
                found = False
            previous_line = line

        if found:
            print("No line following the last match")

In [6]:
Nb_I = np.array(Nb_I).astype(float)
Nb_II = np.array(Nb_II).astype(float)
print("Number of snaps is ", len(Nb_I), "equivalent to ", len(Nb_I)*5, " ps.")

Number of snaps is  540 equivalent to  2700  ps.


In [7]:
from tqdm import tqdm

# Since we want to look at Nb-O units, let us pull out 'O2' coordinates from the HISTORY file
search_string = "O2"
found = False

coords_O=[]
for file_path in file_paths:
    with open(file_path, "r") as file:
        total_lines = sum(1 for _ in file)
        file.seek(0)  # Reset the file pointer to the beginning
        progress_bar = tqdm(total=total_lines, desc="Processing")

        for line in file:
            progress_bar.update(1)  # Update the progress bar
            if search_string in line:
                found = True
            elif found:
            # Print or store the following line
                coords_O.append(np.asarray(line.split()).astype(float))  # or do something else with the line
                found = False

        progress_bar.close()  # Close the prog

coords_O = np.asarray(coords_O).astype(float)
print(np.shape(coords_O))
print(f"Number of O atom in the system is {int(len(coords_O)/len(Nb_I))}")

Processing: 100%|█████████████████████████████████████████████████████████| 53840162/53840162 [01:18<00:00, 684988.24it/s]


(32307120, 3)
Number of O atom in the system is 59828


In [8]:
# Create a list of the box size at each snapshot in the HISTORY file
box = []
for file_path in file_paths:
    with open(file_path, 'r') as file:
        previous_line = None
        found = False
        for line in file:
            if "timestep " in line:
                if previous_line is not None:
                    found = True
                else:
                    found = False
            elif found:
                box.append(float(line.split()[0]))  # Print the line following the match
                found = False
            previous_line = line

        if found:
            print("No line following the last match")
print(len(box))

540


In [11]:
snaps_A0 = len(Nb_I)
NAT = int(len(coords_O)/len(Nb_I))
Nb_O_bond = 2.80

# Create an output xyz file named according to the chosen Nb atoms that were tracked
out1 = open(f'Nb_{Nb_index[0].strip()}_{Nb_index[1].strip()}.xyz','w')

for ind in range(snaps_A0):
    s1 = ind*NAT
    snap_ = coords_O[s1: s1+NAT, :]
    
    d_I = periodic_distance(snap_, Nb_I[ind,:], box[ind])
    d_II = periodic_distance(snap_, Nb_II[ind, :], box[ind])
    indices_I = [i for i,j in enumerate(d_I) if j<Nb_O_bond]
    indices_II = [i for i,j in enumerate(d_II) if j<Nb_O_bond]
    
    print(len(indices_I)+len(indices_II)+2,file=out1)
    print('Frame ',ind,' cell_orig ',-box[ind]*0.5,' ',-box[ind]*0.5,' ',-box[ind]*0.5,\
          ' cell_vec1 ',box[ind],' 0.0 0.0 cell_vec2 0.0 ',box[ind],\
          ' 0.0 cell_vec3 0.0  0.0 ',box[ind],' pbc 1 1 1', file=out1)

    print('Nb ', Nb_I[ind][0], Nb_I[ind][1], Nb_I[ind][2], file = out1)
    print('Nb ', Nb_II[ind][0], Nb_II[ind][1], Nb_II[ind][2], file = out1)
    for k in indices_I:
        print('O ', snap_[k][0], snap_[k][1], snap_[k][2], file = out1)
    for k in indices_II:
        print('O ', snap_[k][0], snap_[k][1], snap_[k][2], file = out1)
out1.close()


You can use this .xyz file to create a movie of the displacement of the Nb-O strucutural units using OVITO or other visualization software. 